In [7]:
import pandas as pd

In [2]:
import requests

token = '37ee3bd571554bda178892bee87a83a5a5c3cc2f'

city = 'Gdansk'

url = f"https://api.waqi.info/feed/{city}/?token={token}"

In [3]:
response = requests.get(url)
data = response.json()

if data["status"] == "ok":
    aqi = data["data"]["aqi"]
    print(f"Indeks jakości powietrza (AQI) w Gdańsku: {aqi}")
else:
    print("Błąd w pobieraniu danych:", data)

Indeks jakości powietrza (AQI) w Gdańsku: 53


In [5]:
SEARCH_URL = f"https://api.waqi.info/search/?token={token}&keyword=Gdansk"

response = requests.get(SEARCH_URL)
stations = response.json()

for station in stations["data"]:
    print(f"Stacja: {station['station']['name']}, AQI: {station['aqi']}, ID: {station['uid']}")

Stacja: Gdansk Srodmiescie, Gdansk, Poland, AQI: 53, ID: 2677
Stacja: Gdansk Wrzeszcz, Gdansk, Poland, AQI: 53, ID: 2684
Stacja: Gdansk Stogi, Gdansk, Poland, AQI: 41, ID: 2678
Stacja: Gdansk Nowy Port, Gdansk, Poland, AQI: 28, ID: 2679
Stacja: Gdansk Szadołki, Gdansk, Poland, AQI: -, ID: 2681
Stacja: Sopot, Gdansk, Poland, AQI: 43, ID: 2682
Stacja: Tczew, Gdansk, Poland, AQI: -, ID: 2683
Stacja: Gdynia Pogorze, Gdansk, Poland, AQI: 48, ID: 2680
Stacja: Gdynia Dabrowa, Gdansk, Poland, AQI: 45, ID: 2685
Stacja: Gdynia Srodmiescie, Gdansk, Poland, AQI: -, ID: 2686
Stacja: gdansko starogardas; Starogard Gdański, Poland, AQI: -, ID: 6566


In [25]:
stations_id = []

for station in stations["data"]:
    stations_id.append(station['uid'])

In [26]:
stations_id

[2677, 2684, 2678, 2679, 2681, 2682, 2683, 2680, 2685, 2686, 6566]

In [24]:
"""HISTORICAL_URL = f"https://api.waqi.info/feed/@2677/?token={token}"
response = requests.get(HISTORICAL_URL)
history = response.json()"""


'HISTORICAL_URL = f"https://api.waqi.info/feed/@2677/?token={token}"\nresponse = requests.get(HISTORICAL_URL)\nhistory = response.json()'

In [34]:
data_list = []
for id in stations_id:
    URL = f"https://api.waqi.info/feed/@{id}/?token={token}"
    response = requests.get(URL)
    data = response.json()

    if data["status"] == "ok":
        # Pobranie indeksu AQI i składników powietrza
        aqi = data["data"]["aqi"]
        components = data["data"]["iaqi"]
        timestamp = data["data"]["time"]["s"]
        location = data["data"]["city"]['name']

        # Przekształcenie do słownika
        air_data = {"Timestamp": timestamp, "Location":location, "AQI": aqi}
        for key, value in components.items():
            air_data[key] = value["v"]

        data_list.append(air_data)
        
    else:
        print("Błąd w pobieraniu danych:", data)

#Stworzenie DataFrame
df3 = pd.DataFrame(data_list)

In [28]:
df

,Timestamp,Location,AQI,co,dew,h,no2,p,pm10,pm25,t,w,wg,o3,so2
0,2025-03-02 18:00:00,"Gdansk Srodmiescie, Gdansk, Poland",55,3.1,2.0,86,3.5,1023.0,14.0,55.0,4.0,8.7,17.4,NaN,NaN
1,2025-03-02 18:00:00,"Gdansk Wrzeszcz, Gdansk, Poland",53,2.6,2.0,86,3.7,1023.0,14.0,53.0,4.0,8.7,17.4,31.0,1.7
2,2025-03-02 18:00:00,"Gdansk Stogi, Gdansk, Poland",47,NaN,2.0,86,1.8,1023.0,12.0,47.0,4.0,8.7,17.4,NaN,1.8
3,2025-03-02 18:00:00,"Gdansk Nowy Port, Gdansk, Poland",31,2.3,2.0,86,1.9,1023.0,8.0,31.0,4.0,8.7,17.4,NaN,1.4
4,2023-12-21 11:00:00,"Gdansk Szadołki, Gdansk, Poland",5,NaN,1.0,93,3.8,984.5,5.0,NaN,2.0,6.4,13.3,22.9,0.7
5,2025-03-02 18:00:00,"Sopot, Gdansk, Poland",50,2.7,2.0,86,2.5,1023.0,12.0,50.0,4.0,8.7,17.4,NaN,1.2
6,2021-07-13 11:00:00,"Tczew, Gdansk, Poland",21,2.6,NaN,72,23.7,986.6,21.0,NaN,24.7,3.5,6.5,NaN,NaN
7,2025-03-02 18:00:00,"Gdynia Pogorze, Gdansk, Poland",53,4.1,NaN,92,2.4,1026.8,13.0,53.0,4.0,10.7,22.5,31.3,1.4
8,2025-03-02 18:00:00,"Gdynia Dabrowa, Gdansk, Poland",45,NaN,2.0,86,2.2,1023.0,11.0,45.0,4.0,8.7,17.4,34.6,1.8
9,2021-06-01 08:00:00,"Gdynia Srodmiescie, Gdansk, Poland",-,NaN,8.0,67,18.1,1016.0,NaN,NaN,14.0,2.0,13.8,NaN,NaN


In [29]:
df.to_csv("air_quality.csv", index=False, encoding="utf-8")

In [33]:
df2

,Timestamp,Location,AQI,co,dew,h,no2,p,pm10,pm25,t,w,wg,o3,so2
0,2025-03-02 19:00:00,"Gdansk Srodmiescie, Gdansk, Poland",55,3.1,1.0,80.0,3.9,1022.5,14.0,55.0,4.0,10.0,17.4,NaN,NaN
1,2025-03-02 19:00:00,"Gdansk Wrzeszcz, Gdansk, Poland",52,2.5,1.0,80.0,3.5,1022.5,13.0,52.0,4.0,10.0,17.4,31.3,1.5
2,2025-03-02 19:00:00,"Gdansk Stogi, Gdansk, Poland",45,NaN,1.0,80.0,1.8,1022.5,12.0,45.0,4.0,10.0,17.4,NaN,1.7
3,2025-03-02 19:00:00,"Gdansk Nowy Port, Gdansk, Poland",31,2.3,1.0,80.0,1.9,1022.5,7.0,31.0,4.0,10.0,17.4,NaN,1.3
4,2023-12-21 11:00:00,"Gdansk Szadołki, Gdansk, Poland",5,NaN,1.0,93.0,3.8,984.5,5.0,NaN,2.0,6.4,13.3,22.9,0.7
5,2025-03-02 19:00:00,"Sopot, Gdansk, Poland",50,2.7,1.0,80.0,2.6,1022.5,12.0,50.0,4.0,10.0,17.4,NaN,1.1
6,2021-07-13 11:00:00,"Tczew, Gdansk, Poland",21,2.6,NaN,72.0,23.7,986.6,21.0,NaN,24.7,3.5,6.5,NaN,NaN
7,2025-03-02 19:00:00,"Gdynia Pogorze, Gdansk, Poland",54,4.1,NaN,92.5,2.4,1027.0,14.0,54.0,3.8,9.5,22.2,31.0,1.5
8,2025-03-02 19:00:00,"Gdynia Dabrowa, Gdansk, Poland",48,NaN,1.0,80.0,2.5,1022.5,11.0,48.0,4.0,10.0,17.4,34.4,1.7
9,2021-06-01 08:00:00,"Gdynia Srodmiescie, Gdansk, Poland",-,NaN,8.0,67.0,18.1,1016.0,NaN,NaN,14.0,2.0,13.8,NaN,NaN


In [35]:
df3

,Timestamp,Location,AQI,co,dew,h,no2,p,pm10,pm25,t,w,wg,o3,so2
0,2025-03-02 21:00:00,"Gdansk Srodmiescie, Gdansk, Poland",52,3.0,2.0,86,2.7,1022.0,12.0,52.0,4.0,7.7,17.4,NaN,NaN
1,2025-03-02 21:00:00,"Gdansk Wrzeszcz, Gdansk, Poland",53,2.6,2.0,86,3.6,1022.0,14.0,53.0,4.0,7.7,17.4,30.1,1.6
2,2025-03-02 21:00:00,"Gdansk Stogi, Gdansk, Poland",52,NaN,2.0,86,7.8,1022.0,17.0,52.0,4.0,7.7,17.4,NaN,3.0
3,2025-03-02 21:00:00,"Gdansk Nowy Port, Gdansk, Poland",31,2.3,2.0,86,2.0,1022.0,7.0,31.0,4.0,7.7,17.4,NaN,1.3
4,2023-12-21 11:00:00,"Gdansk Szadołki, Gdansk, Poland",5,NaN,1.0,93,3.8,984.5,5.0,NaN,2.0,6.4,13.3,22.9,0.7
5,2025-03-02 21:00:00,"Sopot, Gdansk, Poland",49,2.7,2.0,86,2.5,1022.0,12.0,49.0,4.0,7.7,17.4,NaN,1.0
6,2021-07-13 11:00:00,"Tczew, Gdansk, Poland",21,2.6,NaN,72,23.7,986.6,21.0,NaN,24.7,3.5,6.5,NaN,NaN
7,2025-03-02 21:00:00,"Gdynia Pogorze, Gdansk, Poland",52,4.1,NaN,94,2.6,1026.7,13.0,52.0,3.8,10.0,21.0,29.9,1.3
8,2025-03-02 21:00:00,"Gdynia Dabrowa, Gdansk, Poland",48,NaN,2.0,86,2.6,1022.0,11.0,48.0,4.0,7.7,17.4,33.1,1.9
9,2021-06-01 08:00:00,"Gdynia Srodmiescie, Gdansk, Poland",-,NaN,8.0,67,18.1,1016.0,NaN,NaN,14.0,2.0,13.8,NaN,NaN
